In [48]:
import pandas as pd
import numpy as np
import os
import json

In [83]:
DAY_MAP = {"M":0, "T":1, "W":2, "R":3, "F":4}

def time_to_minutes(t):
    """Convert '02:30PM' → minutes since midnight, handles TBA."""
    if t == "TBA" or pd.isna(t):
        return None
    t = t.strip()
    ampm = t[-2:]
    hh, mm = map(int, t[:-2].split(":"))
    if ampm == "PM" and hh != 12:
        hh += 12
    if ampm == "AM" and hh == 12:
        hh = 0
    return hh*60 + mm


In [84]:
df = pd.read_csv("F24-Schedule-ECE.csv")

df = df.rename(columns={
    "COURSE":"course",
    "SEC":"sec",
    "INSTRUCTOR":"instructor",
    "BLDG/ROOM":"room",
    "DAYS":"days",
    "BEGIN":"begin",
    "END":"end",
    "UNITS":"units"
})

print("Raw rows:", len(df))
df.head()


Raw rows: 140


,course,COURSE TITLE,units,sec,MINI,days,begin,end,LOCATION,room,MODE,instructor
0,18021,Introduction to Printed Circuit Boards Fabrica...,3,A1,Y,R,08:00AM,09:20AM,Pittsburgh,ANS B10,In-person,"Bain, James"
1,18021,Introduction to Printed Circuit Boards Fabrica...,3,A2,Y,R,08:00AM,09:20AM,Pittsburgh,ANS B10,In-person,"Bain, James"
2,18050,Study Abroad,0,A,NaN,TBA,TBA,TBA,Pittsburgh,DNM,In-person,"Bain, James"
3,18095,Getting Started in Electronics: An Experientia...,9,A,NaN,MW,11:00AM,12:20PM,Pittsburgh,HH A104,In-person,"Zajdel, Tom; Youssfi, Ziad"
4,18095,Getting Started in Electronics: An Experientia...,9,B,NaN,MW,02:00PM,03:20PM,Pittsburgh,HH A104,In-person,"Zajdel, Tom; Youssfi, Ziad"


In [85]:
expanded = []

for _, row in df.iterrows():

    # Skip TBA times
    if row["begin"] == "TBA":
        continue

    # Expand days
    for d in str(row["days"]):
        if d not in DAY_MAP:
            continue

        expanded.append({
            "course": str(row["course"]),
            "sec": str(row["sec"]),
            "instructor": str(row["instructor"]),
            "room": str(row["room"]),
            "day": DAY_MAP[d],
            "start": time_to_minutes(row["begin"]),
            "end": time_to_minutes(row["end"]),
            "units": row["units"]
        })

expanded = pd.DataFrame(expanded)
expanded = expanded.dropna(subset=["start","end"])
expanded = expanded.sort_values(["course","sec","day","start"])

print("Expanded rows:", len(expanded))
expanded.head()


Expanded rows: 209


,course,sec,instructor,room,day,start,end,units
0,18021,A1,"Bain, James",ANS B10,3,480,560,3
1,18021,A2,"Bain, James",ANS B10,3,480,560,3
2,18095,A,"Zajdel, Tom; Youssfi, Ziad",HH A104,0,660,740,9
3,18095,A,"Zajdel, Tom; Youssfi, Ziad",HH A104,2,660,740,9
4,18095,B,"Zajdel, Tom; Youssfi, Ziad",HH A104,0,840,920,9


In [86]:
sections = (
    expanded[["course","sec","instructor","room","units"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

sections_list = sections.to_dict(orient="records")

print("Number of sections:", len(sections_list))


Number of sections: 131


In [87]:
slots = expanded[["course","sec","day","start","end"]].to_dict(orient="records")

print("Number of time slots:", len(slots))


Number of time slots: 209


In [88]:
os.makedirs("build", exist_ok=True)

with open("build/sections.json","w") as f:
    json.dump(sections_list, f, indent=2)

with open("build/slots.json","w") as f:
    json.dump(slots, f, indent=2)

print("Saved sections.json and slots.json")


Saved sections.json and slots.json


In [89]:
N = len(sections_list)
var_index = { (s["course"], s["sec"]): i for i,s in enumerate(sections_list) }

print("Total variables:", N)


Total variables: 131


In [90]:
# -------------------------------------------
# Save outputs
# -------------------------------------------

os.makedirs("build", exist_ok=True)

with open("build/sections.json","w") as f:
    json.dump(sections_list, f, indent=2)

with open("build/slots.json","w") as f:
    json.dump(slots, f, indent=2)

print("Saved sections.json and slots.json")

Saved sections.json and slots.json


In [91]:
import pandas as pd
import os

MAX_VARS = 50  # or 40/48 if you want, but must be <= 50 for TN1

in_path = "build/qubo_sparse.csv"
out_path = "build/qubo_sparse_tn1.csv"

df = pd.read_csv(in_path)

# keep only terms involving variables < MAX_VARS
mask = (df["i"] < MAX_VARS) & (df["j"] < MAX_VARS)
df_small = df[mask].copy()

print("Original QUBO vars up to index:", int(max(df["i"].max(), df["j"].max())))
print("Reduced QUBO uses vars in [0, %d)" % MAX_VARS)
print("Original rows:", len(df), "→ Reduced rows:", len(df_small))

df_small.to_csv(out_path, index=False)
print("Wrote reduced QUBO to", out_path)


Original QUBO vars up to index: 31
Reduced QUBO uses vars in [0, 50)
Original rows: 86 → Reduced rows: 86
Wrote reduced QUBO to build/qubo_sparse_tn1.csv
